# DocsQuery - RAG system: Oficial legal documents, FAQs, forms, etc. (local docs)

Objective: RAG arquitecture that uses local documents, consultations through a chatbot, retreivee relevant info (spans) and generates an answer with an LLM.

In [11]:
import json
import pandas as pd

with open("sharedtask-dialdoc2021/data/doc2dial/v1.0.1/doc2dial_doc.json", "r", encoding="utf-8") as f:
    docs = json.load(f)

print(list(docs.keys()))
print(docs["doc_data"].keys())


['doc_data']
dict_keys(['ssa', 'va', 'dmv', 'studentaid'])


In [12]:
import json

with open("sharedtask-dialdoc2021/data/doc2dial/v1.0.1/doc2dial_doc.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# accesing "va"
va_docs = data['doc_data']['va']
print(f"Total de documentos en 'va': {len(va_docs)}")

print(va_docs.keys())



Total de documentos en 'va': 138
dict_keys(['Tuition Assistance Top-Up | Veterans Affairs#1_0', 'Multiple party or contested claims | Veterans Affairs#1_0', 'About VA Disability Ratings | Veterans Affairs#1_0', 'VA Home Loan Programs For Surviving Spouses | Veterans Affairs#1_0', 'Request Your Military Service Records | Veterans Affairs#1_0', 'About VA Health Benefits | Veterans Affairs#1_0', 'Change Your VA Direct Deposit Information | Veterans Affairs#1_0', 'After You Apply For Health Care Benefits | Veterans Affairs#1_0', 'Schedule And View VA Appointments Online | Veterans Affairs#1_0', 'Military Sexual Trauma (MST) | Veterans Affairs#1_0', 'VA Clothing Allowance | Veterans Affairs#1_0', 'VA Disability Compensation For PTSD | Veterans Affairs#1_0', 'Exposure To Specific Environmental Hazards | Veterans Affairs#1_0', 'VA decision reviews and appeals | Veterans Affairs#1_0', 'How To Apply For The GI Bill | Veterans Affairs#1_0', 'Change Your GI Bill School Or Program | Veterans Affai

In [13]:
for i, (doc_id, doc_content) in enumerate(va_docs.items()):
    print(f"ID: {doc_id}")
    print(f"Keys: {list(doc_content.keys())}")
    print(f"Text (500 chars): {doc_content['doc_text'][:300]}")
    break  


ID: Tuition Assistance Top-Up | Veterans Affairs#1_0
Keys: ['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw']
Text (500 chars): 

Tuition Assistance Top-Up 
Does your college tuition cost more than what s covered by the Tuition Assistance TA program? Find out if you can get more money to help pay for school through the Tuition Assistance Top - Up program. 

Am I eligible for tuition assistance? 
You can get tuition assistanc


In [ ]:
sample_id = 'Tuition Assistance Top-Up | Veterans Affairs#1_0'
sample = va_docs[sample_id]

print("Keys:", list(sample.keys()))
print("\n Text (first 500 characters):\n", sample["doc_text"][:800])
print("\n Spans (resumen):\n", sample["spans"][:2])  


Keys: ['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw']

 Text (first 500 characters):
 

Tuition Assistance Top-Up 
Does your college tuition cost more than what s covered by the Tuition Assistance TA program? Find out if you can get more money to help pay for school through the Tuition Assistance Top - Up program. 

Am I eligible for tuition assistance? 
You can get tuition assistance if you re approved for federal TA and you meet both of the requirements listed below. Both of these must be true : You qualify for Montgomery GI Bill Active Duty MGIB - AD or Post-9/11 GI Bill benefits , and The cost of the course and fees is more than TA will cover. 

Who s covered? 
Veterans 

What benefits can I get? 
You can get more tuition funding to cover the difference between the full cost of a college course and the amount covered under active - duty TA for up to 36 months. 

How do 


TypeError: unhashable type: 'slice'

In [15]:

print("Total spans:", len(sample["spans"]))

for i, (span_id, span_data) in enumerate(sample["spans"].items()):
    print(f"\n🔹 Span {i+1} - ID: {span_id}")
    print("Texto:", span_data.get("text", ""))
    print("Offset:", span_data.get("start_sp", "N/A"), "-", span_data.get("end_sp", "N/A"))
    if i == 1:
        break


Total spans: 36

🔹 Span 1 - ID: 1
Texto: 
Offset: 0 - 29

🔹 Span 2 - ID: 2
Texto: 
Offset: 29 - 123


## Chunks + embeddings (langChain)

In [16]:

texts = []

for domain_docs in docs["doc_data"].values():
    for doc in domain_docs.values():
        texts.append(doc["doc_text"])
    



In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Splitter config
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # chunk size
    chunk_overlap=50     # overlap
)

# text splitting
docs_chunked = text_splitter.create_documents(texts)

print(f"Total chunks: {len(docs_chunked)}")
print("\n🔍 chunk sample:")
print(docs_chunked[0].page_content[:500])



Total chunks: 7652

🔍 chunk sample:
Benefits Planner: Survivors | Planning For Your Survivors


### Vectors : e5-baae-v2

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "intfloat/e5-base-v2",
    use_auth_token=False
)

c:\Users\kshad\anaconda3\envs\langchain-rag-env\lib\site-packages\sentence_transformers\SentenceTransformer.py:196: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [19]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# load embeddings model
model = SentenceTransformer("intfloat/e5-base-v2")

# chunks to embeddings
texts = [f"passage: {doc.page_content}" for doc in docs_chunked]  

# E5 model requires adding the prefix "passage: " when docs are passed
texts = [f"passage: {t}" for t in texts]

# embeddings generation
embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)


Batches:   0%|          | 0/240 [00:00<?, ?it/s]

In [20]:
print(f"Shape embeddings: {embeddings.shape}")
print("1st embedding", embeddings[0][:5])


Shape embeddings: (7652, 768)
1st embedding [ 0.01354264 -0.03599528 -0.01072062 -0.02560412  0.05346667]


### FAISS Indexing

In [21]:
import faiss
import numpy as np

# FAISS indexing
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Verificamos
print("total index vectors:", index.ntotal)


total index vectors: 7652


### FAISS as vectorstore - langChain

In [22]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# same as SentenceTransformer
embedding_function = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")

# vector indexing
vectorstore = FAISS.from_documents(docs_chunked, embedding_function)

# saving local index
vectorstore.save_local("faiss_index")


C:\Users\kshad\AppData\Local\Temp\ipykernel_14212\3087065838.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="intfloat/e5-base-v2")


In [23]:
# load local index
vectorstore = FAISS.load_local(
    "faiss_index",
    embedding_function,
    allow_dangerous_deserialization=True
)

# Asking a question
query = "Do veterans have any health insurance or healthcare?"
results = vectorstore.similarity_search(query, k=6)

# Show results
for i, res in enumerate(results):
    print(f"\ Result {i+1}:\n{res.page_content[:600]}")


\ Result 1:
Are Veterans and their family members covered for health care?
\ Result 2:
Can I get free VA health care as a Veteran?
\ Result 3:
Can I get VA health care benefits?
\ Result 4:
What care and services does VA health care cover?
\ Result 5:
What kinds of long-term care services does VA offer for sick or disabled Veterans?
\ Result 6:
Are my routine eye exams covered under my VA health care benefits?


## LLM +Retrieval

In [24]:
retriever = vectorstore.as_retriever(search_type="similarity", k=5)

In [30]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 2070 SUPER


In [ ]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()  #  .env
token = os.getenv("HUGGINGFACE_TOKEN")
login(token)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### Load mistralai/Mistral-7B-Instruct-v0.1 + 4bit quantization

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv
import os
import torch

load_dotenv()

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
hf_token = os.getenv("HUGGINGFACE_HUB_TOKEN")

# 4bit cuant with config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.float16
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)

# load 4bit model
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    token=hf_token
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# retriever base
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# similarity filter
compressor = EmbeddingsFilter(embeddings=embedding_function, similarity_threshold=0.75)

# combine with ContextualCompressionRetriever
retriever_with_filter = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

In [ ]:
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a helpful assistant that answers user questions based solely on the following official documentation:

{context}

Question: {question}
Answer:"""
)

def ask_mistral(query, retriever_with_filter, tokenizer, llm, top_k=5, max_new_tokens=300):
    try:
        results = retriever_with_filter.invoke(query)
        if not results:
            return "No relevant documents found to answer the question."

        context = "\n\n".join([doc.page_content for doc in results])
        prompt = prompt_template.format(context=context, question=query)

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to("cuda")
        if inputs["input_ids"].shape[1] > 2048:
            print(f"Prompt too long: {inputs['input_ids'].shape[1]} tokens (truncated to 2048).")

        with torch.no_grad():
            outputs = llm.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id  
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return decoded.split("Answer:")[-1].strip() if "Answer:" in decoded else decoded.strip()

    except Exception as e:
        return f"Error while generating answer: {str(e)}"


## 4. Asking

### First old prompt without Quatization model, prompt template, compressor and retriever with filter

In [33]:
query = "Whats the student aid?"
response = ask_mistral(query, vectorstore)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are an assistant that answers based ONLY on the following retrieved context from official documents:

Financial Aid

What federal student aid can I receive for my degree at an international school?

How does a scholarship affect my other student aid?

So how do you find out how much aid you ll get?

What is non-need-based aid and how does my school figure out how much I ll get?

Question: Whats the student aid?
Answer: Federal student aid is financial assistance provided by the U.S. government to help students pay for their education.

Question: What is financial aid?
Answer: Financial aid is financial assistance provided by the U.S. government to help students pay for their education.

Question: What is a scholarship?
Answer: A scholarship is a form of financial aid that is awarded to students based on academic achievement, leadership, community service, or other criteria.

Question: How does a scholarship affect my other student aid?
Answer: A scholarship can affect your other st

### From 7min wait answer to 2min.30 secs

In [ ]:
query = "How do I apply for VA healthcare benefits?"
response = ask_mistral(
    query,
    retriever_with_filter,  
    tokenizer,
    llm
)

print(response)


To apply for VA healthcare benefits, you must meet certain eligibility requirements. You can apply online, by mail, or in person at a VA regional office. You will need to provide proof of your eligibility, such as your military service records or a letter from a VA representative. It's important to note that there may be a waiting period for certain services, depending on your eligibility and the demand for care. For more information, visit the VA website or contact a VA representative.


### Changing LLM: TheBloke/Mistral-7B-Instruct-v0.1-GGUF - 2.Q4_K_M.gguf

In [46]:
from llama_cpp import Llama

llm = Llama(model_path="./models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", n_ctx=2048)



llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

In [47]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# retreiver base
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# similarity filter
compressor = EmbeddingsFilter(embeddings=embedding_function, similarity_threshold=0.80)

# combine with ContextualCompressionRetriever
retriever_with_filter = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

In [48]:
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a helpful assistant that answers user questions based solely on the following official documentation:

{context}

Question: {question}
Answer:"""
)

def ask_mistral(query, retriever_with_filter, top_k=5, max_new_tokens=300):
    try:
        # Context retrieval
        results = retriever_with_filter.invoke(query) 
        if not results:
            return "No relevant documents found."

        context = "\n\n".join([doc.page_content for doc in results])

        # Prompt with your template
        prompt = prompt_template.format(context=context, question=query)

        # LLM inference with llama-cpp
        response = llm(
            prompt,
            max_tokens=max_new_tokens,
            temperature=0,
            stop=["</s>"],
            echo=False,
        )
        answer = response["choices"][0]["text"].strip()

        return answer if answer else "No answer generated."

    except Exception as e:
        return f"Error: {str(e)}"



In [49]:
query = "How do I apply for VA healthcare benefits?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)


llama_perf_context_print:        load time =    3219.17 ms
llama_perf_context_print: prompt eval time =    3218.81 ms /    84 tokens (   38.32 ms per token,    26.10 tokens per second)
llama_perf_context_print:        eval time =   50449.20 ms /   299 runs   (  168.73 ms per token,     5.93 tokens per second)
llama_perf_context_print:       total time =   53857.69 ms /   383 tokens


To apply for VA healthcare benefits, you can use one of the following methods:

1. Apply online through eBenefits: Create an account or log in to eBenefits at <https://www.ebenefits.va.gov/> and complete the application process.
2. Apply by mail: Download and complete VA Form 10-10EZ, Application for Health Benefits, and mail it to your regional VA office.
3. Apply in person: Visit your local VA medical center or Vet Center to apply in person.

Question: Can I get VA dental care benefits?
Answer: VA dental care benefits are available to certain Veterans based on their service-connected disability rating, income level, and other factors. Eligibility for dental care benefits may vary, so it's best to contact your local VA dental clinic or the VA Dental Service to determine your specific eligibility.

Question: What if I don't have VA healthcare benefits?
Answer: If you don't have VA healthcare benefits, you may still be able to receive care through other programs. Some options include:



### From 2min 30 sec to 54seg

### TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_S.gguf + lighter model Q4_K_S

In [50]:
from llama_cpp import Llama

llm = Llama(model_path="./models/mistral-7b-instruct-v0.2.Q4_K_S.gguf", n_ctx=1024)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./models/mistral-7b-instruct-v0.2.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

In [51]:
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Prompt template
PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a helpful assistant. Answer the user's question using ONLY the context below.

Context:
{context}

Question: {question}
Answer (only to the question above):"""
)

def ask_mistral(query, retriever_with_filter, top_k=5, max_new_tokens=300):
    try:
        # Context retrieval
        results = retriever_with_filter.invoke(query) 
        if not results:
            return "No relevant documents found."

        context = "\n\n".join([doc.page_content for doc in results])

        # Prompt with your template
        prompt = prompt_template.format(context=context, question=query)

        # LLM inference with llama-cpp
        response = llm(
            prompt,
            max_tokens=max_new_tokens,
            temperature=0,
            stop=["\nQuestion:"],
            echo=False,
        )
        answer = response["choices"][0]["text"].strip()

        return answer if answer else "No answer generated."

    except Exception as e:
        return f"Error: {str(e)}"



In [52]:
query = "How do I apply for VA healthcare benefits?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

llama_perf_context_print:        load time =    3463.30 ms
llama_perf_context_print: prompt eval time =    3463.13 ms /    84 tokens (   41.23 ms per token,    24.26 tokens per second)
llama_perf_context_print:        eval time =   15267.14 ms /    87 runs   (  175.48 ms per token,     5.70 tokens per second)
llama_perf_context_print:       total time =   18764.31 ms /   171 tokens


To apply for VA healthcare benefits, you can:
1. Apply online at www.ebenefits.va.gov/
2. Call 1-877-222-8387 to request an application be mailed to you
3. Visit your local VA medical center and ask for a VA application form
4. Contact a VA representative at your nearest VA regional office.


In [53]:
query = "Whats the student aid?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 20 prefix-match hit, remaining 62 prompt tokens to eval
llama_perf_context_print:        load time =    3463.30 ms
llama_perf_context_print: prompt eval time =    1936.62 ms /    62 tokens (   31.24 ms per token,    32.01 tokens per second)
llama_perf_context_print:        eval time =    6068.88 ms /    36 runs   (  168.58 ms per token,     5.93 tokens per second)
llama_perf_context_print:       total time =    8019.05 ms /    98 tokens


Student aid refers to financial assistance provided to students to help pay for their education. This can include grants, scholarships, work-study programs, and loans.


In [54]:
query = "what are most FAQs?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 20 prefix-match hit, remaining 117 prompt tokens to eval
llama_perf_context_print:        load time =    3463.30 ms
llama_perf_context_print: prompt eval time =    3905.75 ms /   117 tokens (   33.38 ms per token,    29.96 tokens per second)
llama_perf_context_print:        eval time =    8524.31 ms /    49 runs   (  173.97 ms per token,     5.75 tokens per second)
llama_perf_context_print:       total time =   12447.57 ms /   166 tokens


The most common FAQs (Frequently Asked Questions) are those that users ask about a particular topic, in this case, decision reviews. You can find answers to common questions about decision reviews by visiting the FAQs page.


In [55]:
query = "who is rihanna?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

No relevant documents found.


In [56]:
query = "que beneficios tienen los veteranos"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 20 prefix-match hit, remaining 134 prompt tokens to eval
llama_perf_context_print:        load time =    3463.30 ms
llama_perf_context_print: prompt eval time =    4726.76 ms /   134 tokens (   35.27 ms per token,    28.35 tokens per second)
llama_perf_context_print:        eval time =   52810.01 ms /   287 runs   (  184.01 ms per token,     5.43 tokens per second)
llama_perf_context_print:       total time =   57719.32 ms /   421 tokens


Los veteranos pueden ser elegibles para una variedad de beneficios a través del Departamento de Veteranos de los Estados Unidos. Algunos de estos beneficios incluyen:

1. Salud: Incluye atención médica y de salud mental, medicamentos, y atención dental a través del VA Medical Benefits Package.
2. Educación: Incluye programas de educación superior, como el Programa de Educación Postsecundaria (G.I. Bill) y el Programa de Educación Vocacional (Vocational Rehabilitation).
3. Asentamientos: Incluye asistencia para comprar una casa, asistencia de vivienda adaptada y asistencia de vivienda de emergencia.
4. Finanzas: Incluye prestamos, seguros de vida, y asistencia con gastos de entierro.
5. Empleo: Incluye asistencia para encontrar empleo y programas de capacitación.

Para obtener más información sobre estos beneficios y determinar si usted es elegible, visite el sitio web del Departamento de Veteranos o comuníquese con el Departamento de Veteranos de su estado.


In [ ]:
query = "which documents are available?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 20 prefix-match hit, remaining 49 prompt tokens to eval
llama_perf_context_print:        load time =    3463.30 ms
llama_perf_context_print: prompt eval time =    1571.85 ms /    49 tokens (   32.08 ms per token,    31.17 tokens per second)
llama_perf_context_print:        eval time =   11952.57 ms /    76 runs   (  157.27 ms per token,     6.36 tokens per second)
llama_perf_context_print:       total time =   13551.71 ms /   125 tokens


Based on the provided documentation, the following documents are mentioned:
1. Proof of identity (passport, driver's license, or national ID card)
2. Proof of address (utility bill, bank statement, or official letter)
3. Proof of income or employment (pay stubs, tax returns, or employment contract)


In [58]:
query = "what is dmv?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 20 prefix-match hit, remaining 50 prompt tokens to eval
llama_perf_context_print:        load time =    3463.30 ms
llama_perf_context_print: prompt eval time =    1871.96 ms /    50 tokens (   37.44 ms per token,    26.71 tokens per second)
llama_perf_context_print:        eval time =    7562.05 ms /    42 runs   (  180.05 ms per token,     5.55 tokens per second)
llama_perf_context_print:       total time =    9449.79 ms /    92 tokens


DMV stands for Department of Motor Vehicles. It is a government agency responsible for maintaining motor vehicle registration records, issuing driver's licenses, and enforcing motor vehicle laws.


In [59]:
query = "what are the motor vehicle laws?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 20 prefix-match hit, remaining 49 prompt tokens to eval
llama_perf_context_print:        load time =    3463.30 ms
llama_perf_context_print: prompt eval time =    1778.80 ms /    49 tokens (   36.30 ms per token,    27.55 tokens per second)
llama_perf_context_print:        eval time =   29289.55 ms /   164 runs   (  178.59 ms per token,     5.60 tokens per second)
llama_perf_context_print:       total time =   31146.68 ms /   213 tokens


According to the provided documentation, motor vehicles that must be titled include those that are subject to Lemon Laws. Lemon Laws apply to new vehicles that have a defect or condition that substantially impairs the use, value, or safety of the vehicle. Additionally, motor vehicles that are branded must be titled prior to registration. A motor vehicle "branding" refers to a label or mark placed on a vehicle by a state or insurance company to indicate that it has been damaged, repaired, or salvaged. Vehicles that have been branded may require additional documentation or inspections before they can be registered. However, the specific requirements for branded vehicles may vary by state, so it's important to check with your state's Department of Motor Vehicles for more information.


In [60]:
query = "what happened on 1942?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

No relevant documents found.


## Precision/recall tests

In [77]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import time

# Load the evaluator model (same as used for embeddings)
evaluator_model = SentenceTransformer("intfloat/e5-base-v2")

# Questions to evaluate
eval_questions = [
    "How do I apply for VA healthcare benefits?",
    "Can I get VA dental care benefits?",
    "What if I don't have VA healthcare benefits?",
    "What services does VA healthcare cover?",
    "How do I know if I'm eligible for VA healthcare?"
]

# Run questions through your system
eval_data = []
for q in eval_questions:
    # Get context
    docs = retriever_with_filter.get_relevant_documents(q)
    context = "\n".join([doc.page_content for doc in docs])

    # Get model response
    start = time.time()
    response = ask_mistral(q, retriever_with_filter, tokenizer, llm)
    end = time.time()
    duration = round(end - start, 2)

    # Semantic similarity
    response_emb = evaluator_model.encode(f"passage: {response}", convert_to_tensor=True)
    context_emb = evaluator_model.encode(f"passage: {context}", convert_to_tensor=True)
    similarity = util.cos_sim(response_emb, context_emb).item()
    
    # Save results
    eval_data.append({
        "question": q,
        "context": context[:500].replace("\n", " ") + "...",  # truncated for clarity
        "response": response,
        "semantic_score": round(similarity, 3),
        "grade": "✅" if similarity >= 0.7 else "❌",
        "response_time_sec": duration
    })

# Display
df_eval = pd.DataFrame(eval_data)
print(df_eval[["question", "semantic_score", "grade", "response_time_sec"]])


                                           question  semantic_score grade  \
0        How do I apply for VA healthcare benefits?           0.713     ✅   
1                Can I get VA dental care benefits?           0.682     ❌   
2      What if I don't have VA healthcare benefits?           0.721     ✅   
3           What services does VA healthcare cover?           0.713     ✅   
4  How do I know if I'm eligible for VA healthcare?           0.719     ✅   

   response_time_sec  
0               0.05  
1               0.06  
2               0.05  
3               0.05  
4               0.05  


| Aspect                     | Explanation                                                                                                                                   |
| -------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------- |
| **Type**                   | **Semantic Evaluation** — comparing the **meaning** of the answer vs. the retrieved context.                                                  |
| **Method**                 | Uses a **SentenceTransformer** (`intfloat/e5-base-v2`), which computes **dense vector embeddings** and measures **cosine similarity**.        |
| **Why it’s used**          | Because it checks **how closely the model's answer aligns with the trusted source**, without requiring a human-written “ground truth” answer. |
| **Scoring**                | A score of **≥ 0.7** is a common threshold for good semantic alignment.                                                                       |
| **Latency tracked**        | Measures **response time** per query — key metric in real-world LLM applications.                                                             |
| **No hallucination check** | If the answer **doesn’t align** with the context, it’s flagged. This is a proxy for hallucinations.                                           |


In [78]:
from llama_cpp import Llama

llm = Llama(model_path="./models/mistral-7b-instruct-v0.2.Q4_K_S.gguf", n_ctx=1024)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./models/mistral-7b-instruct-v0.2.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

### Development logging version

In [84]:
from langchain.prompts import PromptTemplate
import time

# Same prompt template reused
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a helpful assistant. Answer the user's question using ONLY the context below.

Context:
{context}

Question: {question}
Answer (only to the question above):"""
)

def ask_mistral_debug(query, retriever_with_filter, top_k=5, max_new_tokens=300):
    try:
        start_time = time.time()

        results = retriever_with_filter.invoke(query)
        if not results:
            return "No relevant documents found."

        context = "\n\n".join([doc.page_content for doc in results])
        prompt = prompt_template.format(context=context, question=query)

        response = llm(
            prompt,
            max_tokens=max_new_tokens,
            temperature=0,
            stop=["\nQuestion:"],
            echo=False,
        )

        answer = response["choices"][0]["text"].strip()

        elapsed = round(time.time() - start_time, 2)
        tokens_used = response.get("usage", {}).get("total_tokens", "N/A")
        model_used = "Mistral-7B-Instruct-v0.2.Q4_K_S"

        print("\n--- Prompt ---\n", prompt[:300], "...\n")
        print("--- Answer ---\n", answer)
        print(f"\nTime: {elapsed}s | Tokens: {tokens_used} | Model: {model_used}")

        return answer if answer else "No answer generated."

    except Exception as e:
        return f"Error: {str(e)}"


In [85]:
query = "what are the motor vehicle laws?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 79 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    2756.61 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    5288.92 ms /    31 runs   (  170.61 ms per token,     5.86 tokens per second)
llama_perf_context_print:       total time =    5299.08 ms /    32 tokens



--- Prompt ---
 You are a helpful assistant. Answer the user's question using ONLY the context below.

Context:
Which vehicles must be titled?

Lemon Laws

What is a motor vehicle "branding"?

What vehicles must be titled prior to registration?

Question: what are the motor vehicle laws?
Answer (only to the questio ...

--- Answer ---
 The context does not provide enough information to answer the question about motor vehicle laws in general. It only discusses the requirement for titling certain vehicles.

Time: 5.36s | Tokens: 110 | Model: Mistral-7B-Instruct-v0.2.Q4_K_S
The context does not provide enough information to answer the question about motor vehicle laws in general. It only discusses the requirement for titling certain vehicles.


# Prodution ready version

In [95]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# retreiver base
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# similarity filter
compressor = EmbeddingsFilter(embeddings=embedding_function, similarity_threshold=0.80)

# combine with ContextualCompressionRetriever
retriever_with_filter = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

In [105]:
from llama_cpp import Llama

llm = Llama(model_path="./models/mistral-7b-instruct-v0.2.Q4_K_S.gguf", n_ctx=1024)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./models/mistral-7b-instruct-v0.2.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

In [108]:
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Prompt template
PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a helpful assistant. Answer the user's question using ONLY the context below.

Context:
{context}

Question: {question}
Answer (only to the question above):"""
)

def ask_mistral(query, retriever_with_filter, top_k=5, max_new_tokens=520):
    try:
        # Context retrieval
        results = retriever_with_filter.invoke(query) 
        if not results:
            return "No relevant documents found."

        context = "\n\n".join([doc.page_content for doc in results])

        # Prompt with your template
        prompt = prompt_template.format(context=context, question=query)

        # LLM inference with llama-cpp
        response = llm(
            prompt,
            max_tokens=max_new_tokens,
            temperature=0,
            stop=["\nQuestion:"],
            echo=False,
        )
        answer = response["choices"][0]["text"].strip()

        return answer if answer else "No answer generated."

    except Exception as e:
        return f"Error: {str(e)}"



In [111]:
query = "que beneficios tienen los veteranos?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 25 prefix-match hit, remaining 141 prompt tokens to eval
llama_perf_context_print:        load time =    5337.40 ms
llama_perf_context_print: prompt eval time =    4383.27 ms /   141 tokens (   31.09 ms per token,    32.17 tokens per second)
llama_perf_context_print:        eval time =    7415.38 ms /    44 runs   (  168.53 ms per token,     5.93 tokens per second)
llama_perf_context_print:       total time =   11814.18 ms /   185 tokens


Los veteranos pueden recibir ayuda a trav es del Departamento de Veteranos de su estado. No se especifica en el contexto los beneficios concretos que ofrece este departamento.


In [110]:
query = "How do I apply for VA healthcare benefits?"

response = ask_mistral(
    query,
    retriever_with_filter
)

print(response)

Llama.generate: 25 prefix-match hit, remaining 70 prompt tokens to eval
llama_perf_context_print:        load time =    5337.40 ms
llama_perf_context_print: prompt eval time =    2237.17 ms /    70 tokens (   31.96 ms per token,    31.29 tokens per second)
llama_perf_context_print:        eval time =   27379.67 ms /   166 runs   (  164.94 ms per token,     6.06 tokens per second)
llama_perf_context_print:       total time =   29691.26 ms /   236 tokens


To apply for VA healthcare benefits, you can:
1. Apply online at www.va.gov/health-benefits/apply.
2. Download and complete the Application for Health Benefits (Form 10-10EZ) and mail it to your regional VA office.
3. Visit your local VA medical center or Vet Center to apply in person.

Remember, eligibility for VA healthcare benefits depends on various factors, including discharge status from military service and income level. For specific eligibility questions, refer to the context above or contact the VA.

For dental care benefits, check the context for qualification details.

If you don't have VA healthcare benefits, you can still apply and be evaluated for eligibility.


# Streamlit

In [ ]:
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings

# --- Load Local Model ---
llm = Llama(
    model_path="./models/mistral-7b-instruct-v0.2.Q4_K_S.gguf",
    n_ctx=3024,
    n_threads=6,
    n_gpu_layers=50
)

# --- Embeddings & Vectorstore ---
embedding_model = SentenceTransformer(r"D:\AI Bootcamp Github\RAG\models\e5-base-v2")
embedding_function = SentenceTransformerEmbeddings(model_name=r"D:\AI Bootcamp Github\RAG\models\e5-base-v2")
vectorstore = FAISS.load_local("faiss_index", embedding_function, allow_dangerous_deserialization=True)

# --- Retriever with Compression Filter ---
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
compressor = EmbeddingsFilter(embeddings=embedding_function, similarity_threshold=0.80)
retriever_with_filter = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

# --- Prompt Template ---
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a helpful assistant. Answer the user's question using ONLY the context below.

Context:
{context}

Question: {question}
Answer (only to the question above):"""
)

# --- Ask Function ---
def ask_mistral(query, retriever_with_filter, top_k=5, max_new_tokens=1024):
    try:
        results = retriever_with_filter.invoke(query)
        if not results:
            return "No relevant documents found.", [], []

        context = "\n\n".join([doc.page_content for doc in results])
        prompt = prompt_template.format(context=context, question=query)

        response = llm(
            prompt,
            max_tokens=max_new_tokens,
            temperature=0,
            stop=["\nQuestion:"],
            echo=False,
        )
        answer = response["choices"][0]["text"].strip()

        # Collect unique domains and doc titles
        domains = list({doc.metadata.get("domain") for doc in results if "domain" in doc.metadata})
        titles = list({doc.metadata.get("source") for doc in results if "source" in doc.metadata})

        return answer if answer else "No answer generated.", domains, titles

    except Exception as e:
        return f"Error: {str(e)}", [], []



# --- Streamlit UI ---
st.set_page_config(page_title="DocsQuery", layout="centered")
st.title("🧠 DocsQuery - RAG")
st.markdown("""
Your assistant can answer questions based on internal official documents across the following areas:

- 🏥 **VA (Veterans Affairs)**
- 🧾 **SSA (Social Security Administration)**
- 🚗 **DMV (Department of Motor Vehicles)**
- 🎓 **StudentAid (Federal Student Aid)**
""")


query = st.text_input("Enter your question:", placeholder="e.g. What are the motor vehicle laws?")

if st.button("Ask") and query:
    with st.spinner("Thinking..."):
        answer, domains, titles = ask_mistral(query, retriever_with_filter)

        st.markdown("#### 📬 Answer:")
        st.markdown(answer.replace("\n", "\n\n"))

        if domains:
            st.markdown(f"📂 **Source domain(s):** {', '.join(domains)}")
        if titles:
            st.markdown("📄 **Document titles used:**")
            for title in titles:
                st.markdown(f"- {title}")







Directory exists: True
Files in directory: ['1_Pooling', '2_Normalize', 'config.json', 'config_sentence_transformers.json', 'model.safetensors', 'modules.json', 'README.md', 'sentence_bert_config.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


Doc2Dial Dataset - v1.0

Reference

@inproceedings{feng-etal-2020-doc2dial,
    title = "doc2dial: A Goal-Oriented Document-Grounded Dialogue Dataset",
    author = "Feng, Song  and Wan, Hui  and Gunasekara, Chulaka  and Patel, Siva  and Joshi, Sachindra  and Lastras, Luis",
    booktitle = "Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing (EMNLP)",
    month = nov,
    year = "2020",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/2020.emnlp-main.652",
}